# Wikipedia Notable Life Expectancies

# [Notebook 5 of : Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean4_thanak_2022_06_23.ipynb)

## Context

The


## Objective

The

### Data Dictionary

- Feature: Description

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To save/open python objects in pickle file
import pickle

# To help with reading, cleaning, and manipulating data
import pandas as pd
import numpy as np
import re

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

# To play auditory cue when cell has executed, has warning, or has error and set chime theme
import chime

chime.theme("zelda")

<IPython.core.display.Javascript object>

## Data Overview

### Reading, Sampling, and Checking Data Shape

In [2]:
# Reading the dataset
conn = sql.connect("wp_life_expect_clean3.db")
data = pd.read_sql("SELECT * FROM wp_life_expect_clean3", conn)

# Making a working copy
df = data.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 132652 rows and 23 columns.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
0,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21,1994,January,None,None,dancer,ballet designer and director,None,None,None,None,None,None,None,None,86.0,None,United Kingdom of Great Britain and Northern Ireland,None
1,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12,1994,January,None,None,economist,writer,and academic,None,None,None,None,None,None,None,68.0,None,Ireland,None


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df.tail(2)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
132650,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2,2022,June,1980,None,volleyball player,Olympic champion and coach,None,None,None,None,None,None,None,None,69.0,None,Russia,None
132651,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3,2022,June,None,None,engineer,member of the Chinese Academy of Engineering,None,None,None,None,None,None,None,None,86.0,None,"China, People's Republic of",None


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df.sample(5)

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
22103,27,Bill Chipley,", 82, American professional football player .",https://en.wikipedia.org/wiki/Bill_Chipley,2,2002,December,"Boston Yanks, New York Bulldogs",None,professional football player,None,None,None,None,None,None,None,None,None,82.0,None,United States of America,None
108828,9,Abdel Aziz El Mubarak,", 69, Sudanese singer, pneumonia.",https://en.wikipedia.org/wiki/Abdel_Aziz_El_Mubarak,4,2020,February,None,None,singer,pneumonia,None,None,None,None,None,None,None,None,69.0,None,Sudan,None
21377,8,Ken Ashton,", 76, British journalist and trade union leader .",https://en.wikipedia.org/wiki/Ken_Ashton,2,2002,September,general secretary of the National Union of Journalists,None,journalist and trade union leader,None,None,None,None,None,None,None,None,None,76.0,None,United Kingdom of Great Britain and Northern Ireland,None
29068,10,Eugene McCarthy,", 89, American politician, former Democratic United States Senator from Minnesota and presidential primary candidate.",https://en.wikipedia.org/wiki/Eugene_McCarthy,50,2005,December,"1959 1971, and United States Representative 1949 1959",None,politician,former Democratic United States Senator from Minnesota and presidential primary candidate,None,None,None,None,None,None,None,None,89.0,None,United States of America,None
110158,28,Azam Khan,", 93, Pakistani squash player, COVID-19.",https://en.wikipedia.org/wiki/Azam_Khan_(squash_player),6,2020,March,None,None,squash player,COVID 19,None,None,None,None,None,None,None,None,93.0,None,Pakistan,None


<IPython.core.display.Javascript object>

### Checking Data Types, Duplicates, and Null Values

In [5]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132652 entries, 0 to 132651
Data columns (total 23 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   day             132652 non-null  object 
 1   name            132652 non-null  object 
 2   info            132652 non-null  object 
 3   link            132652 non-null  object 
 4   num_references  132652 non-null  object 
 5   year            132652 non-null  int64  
 6   month           132652 non-null  object 
 7   info_parenth    49830 non-null   object 
 8   info_1          35 non-null      object 
 9   info_2          132604 non-null  object 
 10  info_3          62571 non-null   object 
 11  info_4          12605 non-null   object 
 12  info_5          1497 non-null    object 
 13  info_6          216 non-null     object 
 14  info_7          31 non-null      object 
 15  info_8          6 non-null       object 
 16  info_9          1 non-null       object 
 17  info_10   

<IPython.core.display.Javascript object>

#### Loading `nation_map` from Pickle File to Dictionary nation_map

In [6]:
# Load the nation_map
with open("nation_map.pkl", "rb") as f:
    nation_map = pickle.load(f)

<IPython.core.display.Javascript object>

#### Loading `other_species` list from other_species.csv

In [7]:
# Loading other_species list
other_species_df = pd.read_csv("other_species.csv")
other_species = other_species_df["species"].tolist()

<IPython.core.display.Javascript object>

#### Observations:
- With our dataframe, `nation_map`, and `other_species` list loaded, we can proceed to extracting the other features.
- First, we will clean up the divided `info` columns by removing any remaining digits and nationality and country values.
- We will use the same functions from previous notebooks.

#### Function to Save Indices of Rows Matching Regular Expressions Pattern to a List and Print Number of Rows with Match 

In [8]:
# Define a function that takes dataframe, column name, and re pattern as arguments and returns list of indices
# for which column value matches re pattern
def rows_with_pattern(dataframe, column, pattern):
    """
    Takes input of dataframe, column name, and re pattern 
    and returns list of indices for rows that contain match
    for pattern anywhere within value for given column.
    
    dataframe: dataframe
    column: column name
    pattern: re pattern
    """
    index_list = []

    for i in dataframe.index:
        item = dataframe.loc[i, column]
        match = re.search(pattern, item)
        if match:
            index_list.append(i)
    print(
        f"There are {len(index_list)} rows with matching pattern in column '{column}'."
    )
    return index_list

<IPython.core.display.Javascript object>

#### Function to Use rows_with_pattern Function for Multiple Regular Expression Patterns

In [9]:
# Define a function that calls rows_with_pattern function for multiple re patterns
# returning a single list of indices for all rows with any pattern match


def multiple_patterns(dataframe, column, patterns):
    """
    Takes input dataframe, column, and list of re patterns and returns single list 
    of indices for rows in which a match for any pattern is found with re.search
    
    dataframe: dataframe
    column: column name
    patterns: list of re patterns
    """
    rows_combined = []

    # For loop to check each pattern
    for pattern in patterns:

        # List and number of rows matching each pattern
        print(pattern)
        rows_to_check = rows_with_pattern(dataframe, column, pattern)
        print("")

        # Add list for each pattern to combined list
        rows_combined += rows_to_check

    return rows_combined

<IPython.core.display.Javascript object>

### Removing Remaining Digits and Nationality/Country Values from Divided `info` Columns

#### List of Columns to Treat

In [10]:
# List of columns to treat
cols_lst = [
    "info_1",
    "info_2",
    "info_3",
    "info_4",
    "info_5",
    "info_6",
    "info_7",
    "info_8",
    "info_9",
    "info_10",
    "info_11",
    "info_parenth",
]

<IPython.core.display.Javascript object>

#### Removing Digits

In [11]:
# Regular expression for parenthesis and its contents
pattern = r"\d"

# For loop to find indices of rows that have pattern
rows_combined = []
for column in cols_lst:
    dataframe = df[df[column].notna()]
    rows_to_check = rows_with_pattern(dataframe, column, pattern)
    rows_combined += rows_to_check

# Checking a sample of rows
df.loc[rows_combined, :].sample(2)

There are 0 rows with matching pattern in column 'info_1'.
There are 442 rows with matching pattern in column 'info_2'.
There are 2252 rows with matching pattern in column 'info_3'.
There are 1060 rows with matching pattern in column 'info_4'.
There are 69 rows with matching pattern in column 'info_5'.
There are 5 rows with matching pattern in column 'info_6'.
There are 0 rows with matching pattern in column 'info_7'.
There are 0 rows with matching pattern in column 'info_8'.
There are 0 rows with matching pattern in column 'info_9'.
There are 0 rows with matching pattern in column 'info_10'.
There are 0 rows with matching pattern in column 'info_11'.
There are 24403 rows with matching pattern in column 'info_parenth'.


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2
66123,22,Saidi Bwanamdogo,", 45, Tanzanian politician, MP for Chalinze .",https://en.wikipedia.org/wiki/Saidi_Bwanamdogo,2,2014,January,since 2010,None,politician,MP for Chalinze,None,None,None,None,None,None,None,None,45.0,None,Tanzania,None
123712,4,Samuel Ankama,", 63, Namibian politician, MP , complications from COVID-19.",https://en.wikipedia.org/wiki/Samuel_Ankama,5,2021,July,since 2005,None,politician,MP,complications from COVID 19,None,None,None,None,None,None,None,63.0,None,Namibia,None


<IPython.core.display.Javascript object>

In [12]:
# For loop to extract digits
for column in cols_lst:
    for index in set(rows_combined):
        item = df.loc[index, column]
        if item:
            match = re.search(pattern, item)
            if match:
                df.loc[index, column] = re.sub(pattern, "", item)

# Rechecking number and example rows after treatment
# For loop to find indices of rows that have pattern
recheck_rows = []
for column in cols_lst:
    dataframe = df[df[column].notna()]
    rows_to_check = rows_with_pattern(dataframe, column, pattern)
    recheck_rows += rows_to_check

There are 0 rows with matching pattern in column 'info_1'.
There are 0 rows with matching pattern in column 'info_2'.
There are 0 rows with matching pattern in column 'info_3'.
There are 0 rows with matching pattern in column 'info_4'.
There are 0 rows with matching pattern in column 'info_5'.
There are 0 rows with matching pattern in column 'info_6'.
There are 0 rows with matching pattern in column 'info_7'.
There are 0 rows with matching pattern in column 'info_8'.
There are 0 rows with matching pattern in column 'info_9'.
There are 0 rows with matching pattern in column 'info_10'.
There are 0 rows with matching pattern in column 'info_11'.
There are 0 rows with matching pattern in column 'info_parenth'.


<IPython.core.display.Javascript object>

#### Removing Any Remaining Matches with  `nation_map` Keys and Values

In [13]:
%%time

# For loop to extract remaining information matching items in nation_map
for column in cols_lst:
    dataframe = df[df[column].notna()]
    for nationality, country in nation_map.items():
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if nationality in item or country in item:
                    df.loc[index, column] = item.replace(nationality, "").strip().replace(country,'').strip()

CPU times: total: 14min 7s
Wall time: 14min 7s


<IPython.core.display.Javascript object>

#### Observations:
- After that bit of tidying, we can proceed to extracting `known_for` values.
- The bulk of these values should be in `info_2`, according to the Wikipedia defined fields, so we will start there.

## Extracting `known_for` Data
Our goal will be to have some broader categories into which the specific values will fit.  `known_for` is a diverse feature, in that an individual may be known for a long-term role or roles, a specific event, a relationship with another person who is famous, etc.  So, to some extent we will see what we find and adapt as we go.

We will take the following approach:
1. create and check a list of unique values in `info_2` that have a minimum number repeated, sufficient to create sets for each category, but not so exhaustive to be time prohibitive to manually enter.
2. using the pop() method, add each role to it's associated category's set, below.
3. combine the sets for each category into one dictionary.
4. Search for the values in the dictionary and extract the category key value to a new column `known_for`.

In [14]:
# Obtaining values for column and their counts
col_values = df["info_2"].value_counts()

# Creating a list for values that occur more than set number of time
roles_list = [index for index in col_values.index if col_values[index] > 30]

# Checking length of list
print(f"We will examine the top {len(roles_list)} unique values in info_2.")

We will examine the top 448 unique values in info_2.


<IPython.core.display.Javascript object>

In [15]:
# # Using pop to check list items and add to associated dictionary below
# roles_list.pop()

<IPython.core.display.Javascript object>

In [16]:
# Creating sets for each category
politics_govt_law = set(
    [
        "politician",
        "economist",
        "attorney",
        "trade unionist",
        "unionist",
        "aristocrat",
        "diplomat",
        "lawyer",
        "activist",
        "civil rights",
        "federal",
        "judge",
        "political",
        "politic",
        "royal",
        "civil servant",
        "jurist",
        "judge",
        "conservationist",
        "government official",
        "government",
        "barrister",
        "militant",
        "environmentalist",
        "public servant",
        "King",
        "Queen",
        "Princess",
        "Prince",
        "President",
        "Prime Minister",
        "leader",
    ]
)
arts = set(
    [
        "actor",
        "dancer",
        "choreographer",
        "model",
        "television",
        "jazz",
        "singer",
        "composer",
        "conductor",
        "journalist",
        "writer",
        "saxophonist",
        "film director",
        "comedian",
        "photojournalist",
        "poet",
        "actress",
        "film",
        "editor",
        "drummer",
        "producer",
        "songwriter",
        "publisher",
        "author",
        "violinist",
        "rapper",
        "musician",
        "animator",
        "filmmaker",
        "pianist",
        "historian",
        "comic",
        "screenwriter",
        "fashion",
        "designer",
        "guitarist",
        "voice",
        "opera",
        "cinematographer",
        "playwright",
        "sculptor",
        "novelist",
        "photographer",
        "architect",
        "painter",
        "artist",
        "disc jockey",
        "dj",
        "DJ",
        "MC",
        "bridge player",
        "tenor",
        "trombonist",
        "filmmaker",
        "ballerina",
        "bassist",
        "film critic",
        "critic",
        "personality",
        "organist",
        "operatic",
        "lyricist",
        "translator",
        "visual artist",
        "soprano",
        "cellist",
        "broadcaster",
        "chef",
        "literary critic",
        "ballet",
        "illustrator",
        "theatre director",
        "trumpeter",
        "presenter",
        "sportscaster",
        "cartoonist",
        "sportswriter",
    ]
)
sports = set(
    [
        "football",
        "footballer",
        "Olympic",
        "skier",
        "hockey",
        "soccer",
        "cricket",
        "soccer",
        "sprinter",
        "equestrian",
        "gymnast",
        "fencer",
        "chess",
        "wrestler",
        "swimmer",
        "basketball",
        "hurler",
        "sailor",
        "rower",
        "rugby",
        "athlete",
        "golfer",
        "boxer",
        "tennis",
        "cyclist",
        "racing",
        "driver",
        "cricketer",
        "baseball",
        "speedway rider",
        "speedway",
        "rider",
        "badminton",
        "sport shooter",
        "runner",
        "umpire",
        "judoka",
        "volleyball",
        "track and field",
        "track",
        "bobsledder",
        "canoer",
        "bodybuilder",
        "skater",
        "curler",
        "Olympic diver",
        "martial artist",
        "racer",
        "handball",
        "ski jumper",
        "racehorse trainer",
        "racecar driver",
        "hurdler",
        "polo",
        "Olympic shooter",
        "weightlifter",
        "Baseball",
        "mountaineer",
        "jockey",
        "Olympic sports shooter",
        "referee",
    ]
)
sciences = set(
    [
        "engineer",
        "physicist",
        "geologist",
        "psychiatrist",
        "botanist",
        "biologist",
        "anthropologist",
        "astronomer",
        "biochemist",
        "scientist",
        "computer",
        "archaeologist",
        "psychologist",
        "sociologist",
        "physician",
        "chemist",
        "physicist",
        "mathematician",
        "cosmonaut",
        "pediatrician",
        "astronaut",
        "entomologist",
        "cardiologist",
        "doctor",
        "nurse",
        "immunologist",
        "meteorologist",
        "medical researcher",
        "ornithologist",
        "neuroscientist",
        "microbiologist",
        "zoologist",
        "geographer",
        "inventor",
        "geneticist",
        "surgeon",
        "astrophysicist",
        "statistician",
    ]
)
business = set(
    [
        "executive",
        "business",
        "businessman",
        "banker",
        "entrepreneur",
        "real estate developer",
        "restaurateur",
        "businesswoman",
    ]
)
academia = set(
    [
        "scholar",
        "linguist",
        "educator",
        "philosopher",
        "academic",
        "historian",
        "educationalist",
        "philologist",
        "librarian",
        "industrialist",
        "professor",
        "musicologist",
    ]
)
law_enf_military_operator = set(
    [
        "officer",
        "army",
        "Army",
        "general",
        "police",
        "admiral",
        "soldier",
        "Air Force",
        "intelligence",
        "major",
        "lieutenant",
        "admiral",
        "fighter pilot",
        "pilot",
        "naval",
        "Navy",
        "aviator",
    ]
)
religion = set(
    [
        "rabbi",
        "Catholic",
        "priest",
        "Anglican",
        "cardinal",
        "theologian",
        "prelate",
        "Orthodox",
        "Episcopal",
        "bishop",
        "Jesuit",
        "hierarch",
        "Islamic",
        "religious leader",
    ]
)
social = set(["philanthropist", "socialite"])
crime = set(
    ["serial killer", "murderer", "convicted", "terrorist", "mobster", "criminal"]
)
victim = set(["Holocaust survivor", "victim"])
age = set(["supercentenarian", "oldest person"])

<IPython.core.display.Javascript object>

#### Observations:
- We have a good start on `known_for` values for which to search.  Some other roles that have been observed previously were added to the list also.
- Note that roles such as sportswriter and sports broadcaster, though associated with sports, are included in arts, to align with the nature of the work itself.
- Let us combine them into one dictionary, taking care to put arts last to avoid missing values for "martial artist" and to put religion before politics_govt_law so that "leader" comes after "religious leader".
- Then, we can proceed to extract the category to a new column, `known_for`.
- We will proceed left to right, then search for roles embedded in the column value.  In this case, there is a bias to maintain the first role found, in the order of the dictionary, rather than left to right for the column value.

#### Creating `known_for` Dictionary of Category Keys and Specific Role Sets of Values

In [17]:
# Combining separate dictionaries into one dictionary putting arts last
known_for_dict = {
    "age": age,
    "victim": victim,
    "crime": crime,
    "social": social,
    "religion": religion,
    "law_enf_military_operator": law_enf_military_operator,
    "academia": academia,
    "business": business,
    "sciences": sciences,
    "sports": sports,
    "politics_govt_law": politics_govt_law,
    "arts": arts,
    "other_species": other_species,
}

<IPython.core.display.Javascript object>

#### Extracting Category to `known_for` Column

In [18]:
# Initializing known_for column
df["known_for"] = ""

<IPython.core.display.Javascript object>

#### Searching for `known_for` from Left to Right within `info_1`

In [19]:
%%time

# Column to check
column = 'info_1'

# Extract to column
extract_to = 'known_for'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if item.startswith(role):
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df["known_for"]!=""])} category values in known_for column.')
df[df['known_for']!=''].sample(2)

There are 19 category values in known_for column.
CPU times: total: 1.89 s
Wall time: 1.88 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for
103467,26,Edmund Seger,", 82 German Olympic wrestler.",https://en.wikipedia.org/wiki/Edmund_Seger,2,2019,May,None,wrestler,None,None,None,None,None,None,None,None,None,None,82.0,None,Germany,None,sports
61945,1,Basil Soper,", British actor, 74–75.",https://en.wikipedia.org/wiki/Basil_Soper,0,2013,June,None,,None,None,None,None,None,None,None,None,None,None,74.5,None,United Kingdom of Great Britain and Northern Ireland,None,arts


<IPython.core.display.Javascript object>

#### Searching for `known_for` Embedded withing `info_1` Value

In [20]:
%%time

# Column to check
column = 'info_1'

# Extract to column
extract_to = 'known_for'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df["known_for"]!=""])} category values in known_for column.')
df[df['known_for']!=''].sample(2)

There are 28 category values in known_for column.
CPU times: total: 1.86 s
Wall time: 1.86 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for
87054,8,Mohamud Muse Hersi,", Somali politician, 79–80, President of Puntland .",https://en.wikipedia.org/wiki/Mohamud_Muse_Hersi,12,2017,February,,ian,None,President of Puntland,None,None,None,None,None,None,None,None,79.5,None,Somalia,None,politics_govt_law
61945,1,Basil Soper,", British actor, 74–75.",https://en.wikipedia.org/wiki/Basil_Soper,0,2013,June,None,,None,None,None,None,None,None,None,None,None,None,74.5,None,United Kingdom of Great Britain and Northern Ireland,None,arts


<IPython.core.display.Javascript object>

#### Observations:
- Once again, the `info_1` column has provided a small sample on which to test our code, which appears to be working.
- We can move on to extracting values from `info_2`, starting from left to right, then repeating for roles embedded within the column value.

#### Searching for `known_for` from Left to Right within `info_2`

In [21]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if item.startswith(role):
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df["known_for"]!=""])} category values in known_for column.')
df[df['known_for']!=''].sample(2)

There are 99765 category values in known_for column.
CPU times: total: 2min 58s
Wall time: 2min 58s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for
73835,18,Väinö Kuisma,", 80, Finnish Olympic athlete.",https://en.wikipedia.org/wiki/V%C3%A4in%C3%B6_Kuisma,1,2015,February,None,None,athlete,None,None,None,None,None,None,None,None,None,80.0,None,Finland,None,sports
43708,12,Ken Emerson,", 82, Australian cartoonist .",https://en.wikipedia.org/wiki/Ken_Emerson,7,2010,February,,None,,None,None,None,None,None,None,None,None,None,82.0,None,Australia,None,arts


<IPython.core.display.Javascript object>

#### Searching for `known_for` Embedded withing `info_2` Value

In [22]:
%%time

# Column to check
column = 'info_2'

# Extract to column
extract_to = 'known_for'

# For loop to find role in column and extract it as category to extract_to column
for category, category_set in known_for_dict.items():
    for role in category_set:
        dataframe = df[(df[column].notna()) & (df[extract_to]=='')]
        for index in dataframe.index:
            item = df.loc[index, column]
            if item:
                if role in item:
                    df.loc[index, extract_to] = category
                    df.loc[index, column] = item.replace(role, '').strip()

# Checking number of values found and a sample of rows
print(f'There are {len(df[df["known_for"]!=""])} category values in known_for column.')
df[df['known_for']!=''].sample(2)

There are 122727 category values in known_for column.
CPU times: total: 47.2 s
Wall time: 47.2 s


,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for
35970,20,Ann Baumgartner,", 89, American aviator.",https://en.wikipedia.org/wiki/Ann_Baumgartner,5,2008,March,None,None,,None,None,None,None,None,None,None,None,None,89.0,None,United States of America,None,law_enf_military_operator
52983,22,Moisés Kaiman,", 97, Polish-born Mexican rabbi.",https://en.wikipedia.org/wiki/Mois%C3%A9s_Kaiman,2,2012,January,None,None,,None,None,None,None,None,None,None,None,None,97.0,None,Poland,Mexico,religion


<IPython.core.display.Javascript object>

In [23]:
print("dunzo!")
chime.success()

dunzo!


<IPython.core.display.Javascript object>

In [24]:
df[df["info_1"].notna()]

,day,name,info,link,num_references,year,month,info_parenth,info_1,info_2,info_3,info_4,info_5,info_6,info_7,info_8,info_9,info_10,info_11,age,cause_of_death,place_1,place_2,known_for
6832,17,Spiro Agnew,", American politician, 77, 39th Vice President of the United States, leukemia.",https://en.wikipedia.org/wiki/Spiro_Agnew,207,1996,September,None,ian,None,th Vice President of the,leukemia,None,None,None,None,None,None,None,77.0,None,United States of America,None,politics_govt_law
7507,21,Kell Areskoug,", 90 Swedish Olympic sprinter.",https://en.wikipedia.org/wiki/Kell_Areskoug,5,1996,December,None,sprinter,None,None,None,None,None,None,None,None,None,None,90.0,None,Sweden,None,sports
8861,13,George Strugar,", 63. American gridiron football player, lung cancer.",https://en.wikipedia.org/wiki/George_Strugar,0,1997,June,None,gridiron player,lung cancer,None,None,None,None,None,None,None,None,None,63.0,None,United States of America,None,sports
11825,23,Manuel Mejía Vallejo,", 75 Colombian writer.",https://en.wikipedia.org/wiki/Manuel_Mej%C3%ADa_Vallejo,2,1998,July,None,,None,None,None,None,None,None,None,None,None,None,75.0,None,Colombia,None,arts
12317,25,Sir Robin Brook,", 90 British businessman, banker and Olympic fencer.",https://en.wikipedia.org/wiki/Robin_Brook,3,1998,October,None,man,banker and Olympic fencer,None,None,None,None,None,None,None,None,None,90.0,None,United Kingdom of Great Britain and Northern Ireland,None,business
17841,28,Marie Jahoda,", 94 Austrian-British social psychologist.",https://en.wikipedia.org/wiki/Marie_Jahoda,3,2001,April,None,social,None,None,None,None,None,None,None,None,None,None,94.0,None,United Kingdom of Great Britain and Northern Ireland,None,sciences
18119,4,Dipendra,", King of Nepal, 29, suicide.",https://en.wikipedia.org/wiki/Dipendra_of_Nepal,8,2001,June,None,of,None,suicide,None,None,None,None,None,None,None,None,29.0,None,Nepal,None,politics_govt_law
19580,20,Dame Miraka Szászy,", 80. New Zealand Maori leader.",https://en.wikipedia.org/wiki/Mira_Sz%C3%A1szy,21,2001,December,None,,None,None,None,None,None,None,None,None,None,None,80.0,None,New Zealand,New Zealand,politics_govt_law
20388,8,Helen Gilbert,", 80 American artist.",https://en.wikipedia.org/wiki/Helen_Gilbert,4,2002,April,None,,None,None,None,None,None,None,None,None,None,None,80.0,None,United States of America,None,arts
21050,19,Frank Taylor,", 81. English sports journalist.",https://en.wikipedia.org/wiki/Frank_Taylor_(journalist),3,2002,July,None,sports,None,None,None,None,None,None,None,None,None,None,81.0,None,United Kingdom of Great Britain and Northern Ireland,None,arts


<IPython.core.display.Javascript object>

In [26]:
len(df) - 122727

9925

<IPython.core.display.Javascript object>